# Compare Serena's and Shinhan's annotation

## Testing a few cases

* <><>X</> Y</>: give "X Y", "X"
* <><>X</> <>Y</></>: "X Y", "X", "Y"
* <><>X</> <>Y</> <>Z</></>: "X Y Z", "X", "Y", "Z"
* <><><>X</> Y</> <>Z</></>: "X Y Z", "X Y", "Z"

In [67]:
##
# Test functions

def a_iterate(astr, alist):
    
    # See if this string can be parsed into entities
    astr2 = a_check(astr)
    
    # Cannot be further parsed with a_check
    if len(astr2) == 1:
        alist = strip_tags(astr2[0], alist)
        astr_stripped_outer = strip_outer(astr2[0])
        alist = a_iterate(astr_stripped_outer, alist)
    else:
        for i in astr2:
            alist = a_iterate(i, alist)
            
    return alist

# Begin tag: <entsl>
# End tag: </entsl>
# Can be nested
#   e.g., <><>blah</> bleh</>
def a_check(astr):
    entL = [] # entity list
    cBgn = 0  # begin tag count
    iBgn = 0  # begin tag index
    cEnd = 0  # end tag count
    iEnd = 0  # end tag index
    for i in range(len(astr)):
        # End tag
        if astr[i] == "<":
            if astr[i+1] == "/":
                #print("Found end:",astr[iBgn:i])
                cEnd += 1
                
                # Equal number of begin and end tags, bound an entity
                if cEnd == cBgn:
                    # Set end tag index
                    iEnd = i + len("</>")
                    #print("Match end:",astr[iBgn:iEnd])
                    
                    ent = astr[iBgn:iEnd]
                    entL.append(ent)
                    
                    # Reset
                    cBgn = cEnd = iBgn = iEnd = 0
            # Otherwise it is a begin tag
            else:
                #print("Found bgn:",filestr[i:i+10])
                # First begin tag, set begin tag index
                if cBgn == 0:
                    iBgn = i
                cBgn += 1
    return entL

# Rid of all tags
def strip_tags(astr2, alist):
    astr3 = []
    # Rid of all <> and </>
    astr2 = astr2.split("<>")
    for i in astr2:
        i = i.split("</>")
        astr3.extend(i)
    
    #print(astr3)
    alist.append("".join(astr3))
    return alist

# Rid of the outer most tags
def strip_outer(astr2):
    astr4 = astr2[astr2.find(">")+1:astr2.rfind("<")]
    print(astr4)
    return astr4

In [72]:
# Test cases

astr  = "<><><>X</> Y</> <>Z</></>"
alist = []
alust = a_iterate(astr, alist)
alist

<><>X</> Y</> <>Z</>
<>X</> Y
X
Z


['X Y Z', 'X Y', 'X', 'Z']

In [69]:
# Tested
"<><>X</> Y</>"
"<><>X</> <>Y</></>"
"<><>X</> <>Y</> <>Z</></>"
"<><><>X</> Y</> <>Z</></>"

'<><>X</> <>Y</></>'

## Parse the annotated files

In [1]:
from pathlib import Path

data = Path('data')

sgl  = data / "abstract-jasmonicac-set - sgl_1-6.txt"
shs  = data / "abstract-jasmonicac-set - shs_1-6.txt"
ori  = data / ""

In [29]:
# Do 3 things
# 1) lower case everything
# 2) Rid of empty lines
# 3) Rid of new line char
# Return
#   A string with lines joined with one space
def cleanup(file):
    cleaned = []
    filein  = open(file).readlines()
    for i in filein:
        i = i.lower().strip()
        if i != "":
            cleaned.append(i)
    
    return " ".join(cleaned)

In [102]:
def a_iterate(astr, alist): 
    astr2 = a_check(astr)
    if len(astr2) == 1:
        alist = strip_tags(astr2[0], alist)
        astr_stripped_outer = strip_outer(astr2[0])
        alist = a_iterate(astr_stripped_outer, alist)
    else:
        for i in astr2:
            alist = a_iterate(i, alist)
            
    return alist

def a_check(astr):
    entL = [] # entity list
    cBgn = 0  # begin tag count
    iBgn = 0  # begin tag index
    cEnd = 0  # end tag count
    iEnd = 0  # end tag index
    for i in range(len(astr)):
        if astr[i] == "<":
            if astr[i+1] == "/":
                cEnd += 1
                if cEnd == cBgn:
                    iEnd = i + len("</ent>")
                    ent = astr[iBgn:iEnd]
                    entL.append(ent)
                    cBgn = cEnd = iBgn = iEnd = 0
            else:
                if cBgn == 0:
                    iBgn = i
                cBgn += 1
    return entL

# Rid of all tags
def strip_tags(astr2, alist):
    astr3 = []
    astr2 = astr2.split("<ent>")
    for i in astr2:
        i = i.split("</ent>")
        astr3.extend(i)
    alist.append("".join(astr3))
    return alist

# Rid of the outer most tags
def strip_outer(astr2):
    astr4 = astr2[astr2.find(">")+1:astr2.rfind("<")]
    return astr4

def list_to_dict(alist):
    adict = {}
    for i in alist:
        if i not in adict:
            adict[i] = 0
    return adict

In [103]:
sgl_str = cleanup(sgl)
sgl_lst = []
sgl_lst = a_iterate(sgl_str, sgl_lst)
sgl_dct = list_to_dict(sgl_lst)
len(sgl_dct)

136

In [108]:
shs_str = cleanup(shs)
shs_lst = []
shs_lst = a_iterate(shs_str, shs_lst)
shs_dct = list_to_dict(shs_lst)
len(shs_dct)

84

In [110]:
oup = open("ent_compare.txt","w")
oup.write("Entity\tSL\tSS\n")
for i in sgl_dct:
    if i in shs_dct:
        shs_dct[i] = 1
        oup.write(f"{i}\t1\t1\n")
    else:
        oup.write(f"{i}\t1\t0\n")
    
for i in shs_dct:
    if shs_dct[i] == 0:
        oup.write(f"{i}\t0\t1\n")
        #print("Only in SS:",i)
    
oup.close()